<a href="https://colab.research.google.com/github/bradley-Amuj/ML-Group-Final-Task/blob/main/ML_Group_Final_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Questions**

1.The below problem is a muticlass classification problems. This is because, there exists multiple classes(5) in the dataset that are to be classified.

**IMPORTS**

In [1]:
import os
import glob
import h5py

from tensorflow import keras
from PIL import Image
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from tensorflow.keras.optimizers import Adam , SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from tensorflow.keras.utils import to_categorical
from keras import backend as K
import numpy as np


**Creating data generators**

In [2]:
batch_size = 32  #was 16
train_images = '/content/drive/MyDrive/ML_Dataset/FaceShape Dataset/training_set'
test_images = '/content/drive/MyDrive/ML_Dataset/FaceShape Dataset/training_set'

TrainDatagen = ImageDataGenerator(
        preprocessing_function= preprocess_input,
        horizontal_flip = True


)

TestDatagen = ImageDataGenerator(
    preprocessing_function= preprocess_input

)

train_data = TrainDatagen.flow_from_directory(
    train_images,
    target_size = (224,224),
    batch_size =batch_size,
    class_mode = 'categorical'


)

test_data = TestDatagen.flow_from_directory(
    test_images,
    target_size = (224,224),
    batch_size = batch_size,
    class_mode = 'categorical'
)

Found 4000 images belonging to 5 classes.
Found 4000 images belonging to 5 classes.


**Creating the Model**

In [3]:
def build_model():
    input_img = Input(shape=(224,224,3), name='ImageInput')
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)
    
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)
    x = Dropout(0.3)(x)
    
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)
    x = Dropout(0.5)(x)
    
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_1')(x)
    x = BatchNormalization(name='bn3')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_2')(x)
    x = BatchNormalization(name='bn4')(x)
    x = Dropout(0.5)(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_3')(x)
    x = MaxPooling2D((2,2), name='pool4')(x)
    
    
    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dropout(0.7, name='dropout1')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(5, activation='softmax', name='fc3')(x)
    
    model = Model(inputs=input_img, outputs=x)
    return model




model =  build_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ImageInput (InputLayer)      [(None, 224, 224, 3)]     0         
_________________________________________________________________
Conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
Conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
Conv2_1 (SeparableConv2D)    (None, 112, 112, 128)     8896      
_________________________________________________________________
Conv2_2 (SeparableConv2D)    (None, 112, 112, 128)     17664     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0     

Reason for using separable convolutions instead of the normal 2DConvolutions:

1.   It offers fewer trainable weight parameters hence faster training.- This layer performs a spatial convolution on each of the 3 color channels independently before mixing output channels via a pointwise convolution(1x1 convolution) ie This helps in separating learning of the spatial features and learning of the channel wise features.




In [4]:
# opt = RMSprop(lr=0.0001, decay=1e-6)
opt = Adam(lr=0.0001, decay=1e-5)
# opt = SGD()
# es = EarlyStopping(patience=10,min_delta= .05, monitor="val_acc")
chkpt = ModelCheckpoint(filepath='best_model_todate', save_best_only=True, save_weights_only=True)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=opt)

# metric being tracked is accuracy. This is in order to measure how well each images is being correctly classified to its group.

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


**Training the Model**

In [5]:
# Fit the model
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
history = model.fit_generator(
    train_data,              
    steps_per_epoch = train_data.samples//batch_size,
    validation_data = test_data,
    validation_steps = test_data.samples//batch_size,
    epochs = 50,
    callbacks=[chkpt]
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
125/125 [==============================] - 2874s 23s/step - loss: 1.6099 - accuracy: 0.2077 - val_loss: 1.6094 - val_accuracy: 0.2000
Epoch 2/50
125/125 [==============================] - 177s 1s/step - loss: 1.6082 - accuracy: 0.2115 - val_loss: 1.6095 - val_accuracy: 0.2000
Epoch 3/50
125/125 [==============================] - 177s 1s/step - loss: 1.6054 - accuracy: 0.2320 - val_loss: 1.6096 - val_accuracy: 0.2000
Epoch 4/50
125/125 [==============================] - 178s 1s/step - loss: 1.6024 - accuracy: 0.2447 - val_loss: 1.6103 - val_accuracy: 0.2000
Epoch 5/50
125/125 [==============================] - 179s 1s/step - loss: 1.5828 - accuracy: 0.2660 - val_loss: 1.6115 - val_accuracy: 0.2005
Epoch 6/50
125/125 [==============================] - 179s 1s/step - loss: 1.5394 - accuracy: 0.2880 - val_loss: 1.5967 - val_accuracy: 0.2327
Epoch 7/50
125/125 [==============================] - 180s 1s/step - loss: 1.4984 - accuracy: 0.3235 - val_loss: 1.5007 - val_accuracy: 0.35

**Plotting acc vs Val_ acc and loss vs val loss**

In [6]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

import matplotlib.pyplot as plt


epochs = range(len(acc))
plt.plot(epochs,acc,'bo',label='Training acc')
plt.plot(epochs,val_acc,'r',label='Validation acc')
plt.legend()


plt.figure()
plt.plot(epochs,loss,'bo',label='Training loss')
plt.plot(epochs,val_loss,'r',label='Validation loss')

plt.legend()
plt.show()

KeyError: ignored

**Saving model**

In [ ]:
model.save('new_face_classifier.h5')